In [1]:
import numpy as np
import pandas as pd
import networkx as nx

import altair as alt

/home/pi/code/envs/graph/lib/python3.8/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
nodes = [c for c in 'ABCDEFG']

edges = []
for n in range(15):
    start = np.random.choice(nodes)
    end = np.random.choice(list(set(nodes) - {start}))
    edges += [(str(start), str(end))]
    
G = nx.Graph()
G.add_nodes_from(nodes)
G.add_edges_from(edges)

pos = nx.spring_layout(G)

In [3]:
wordlist = "Declarative visualization grammars can accelerate development, facilitate retargeting across platforms, and allow language-level optimizations. However, existing declarative visualization languages are primarily concerned with visual encoding, and rely on imperative event handlers for interactive behaviors. In response, we introduce a model of declarative interaction design for data visualizations. Adopting methods from reactive programming, we model low-level events as composable data streams from which we form higher-level semantic signals. Signals feed predicates and scale inversions, which allow us to generalize interactive selections at the level of item geometry (pixels) into interactive queries over the data domain. Production rules then use these queries to manipulate the visualization’s appearance. To facilitate reuse and sharing, these constructs can be encapsulated as named interactors: standalone, purely declarative specifications of interaction techniques. We assess our model’s feasibility and expressivity by instantiating it with extensions to the Vega visualization grammar. Through a diverse range of examples, we demonstrate coverage over an established taxonomy of visualization interaction techniques.We present Reactive Vega, a system architecture that provides the first robust and comprehensive treatment of declarative visual and interaction design for data visualization. Starting from a single declarative specification, Reactive Vega constructs a dataflow graph in which input data, scene graph elements, and interaction events are all treated as first-class streaming data sources. To support expressive interactive visualizations that may involve time-varying scalar, relational, or hierarchical data, Reactive Vega’s dataflow graph can dynamically re-write itself at runtime by extending or pruning branches in a data-driven fashion. We discuss both compile- and run-time optimizations applied within Reactive Vega, and share the results of benchmark studies that indicate superior interactive performance to both D3 and the original, non-reactive Vega system.We present Vega-Lite, a high-level grammar that enables rapid specification of interactive data visualizations. Vega-Lite combines a traditional grammar of graphics, providing visual encoding rules and a composition algebra for layered and multi-view displays, with a novel grammar of interaction. Users specify interactive semantics by composing selections. In Vega-Lite, a selection is an abstraction that defines input event processing, points of interest, and a predicate function for inclusion testing. Selections parameterize visual encodings by serving as input data, defining scale extents, or by driving conditional logic. The Vega-Lite compiler automatically synthesizes requisite data flow and event handling logic, which users can override for further customization. In contrast to existing reactive specifications, Vega-Lite selections decompose an interaction design into concise, enumerable semantic units. We evaluate Vega-Lite through a range of examples, demonstrating succinct specification of both customized interaction methods and common techniques such as panning, zooming, and linked selection.".split(' ')
for edge in G.edges:
    G.edges[edge]['words'] = np.random.choice(wordlist, size=np.random.randint(8), replace=False)

for node in G.nodes:
    G.nodes[node]['favorite_color'] =  np.random.choice(['blue', 'blue', 'blue', 'yellow'], replace=False)

In [150]:
pos_data = pd.DataFrame.from_dict(pos).transpose().reset_index(
).rename(
    columns={'index': 'node_id', 0:'x', 1:'y'}
)

node_data_flat = pd.DataFrame.from_dict(G.nodes.data())
node_data = node_data_flat.join(pd.json_normalize(node_data_flat.pop(1))
                               ).rename(columns={0:'node_id'})

edge_data = nx.to_pandas_edgelist(G).reset_index(
).rename(
    columns={'index': 'edge_id'}
)


In [5]:
edge_data['wordcount'] = edge_data['words'].apply(len)

In [147]:
edge_data

,edge_id,source,target,words
0,0,A,C,"[Vega-Lite, studies, grammar, extents,, which,..."
1,1,B,F,[]
2,2,B,C,"[for, within]"
3,3,C,G,[a]
4,4,C,D,"[can, synthesizes, low-level]"
5,5,D,F,"[visualization, system, by, range, customized]"
6,6,D,E,"[decompose, appearance., reuse, retargeting, e..."
7,7,D,G,"[present, concerned, our, for]"
8,8,E,F,"[first, events]"
9,9,E,G,"[novel, an, parameterize, To, us, traditional,..."


In [94]:
position_lookup = alt.LookupData(data=pos_data, key='node_id', fields=['x', 'y'])

nodes = alt.Chart(node_data).mark_point(
    size=200, 
).encode(
    x='x:Q',
    y='y:Q',
#    fill='favorite_color'
).transform_lookup(
    lookup='node_id',
    from_=position_lookup
)

labels = nodes.mark_text(
    align='left',
    baseline='middle',
    dx=10,
    dy=-10
).encode(text='node_id')

edges = alt.Chart(edge_data).mark_line().encode(
    x = alt.X('x:Q', axis=None),
    y = alt.Y('y:Q', axis=None),
    x2 = alt.X2('x2:Q'),
    y2 = alt.Y2('y2:Q'),
    size = 'wordcount'
).transform_lookup(
    lookup='source',
    from_=position_lookup
).transform_lookup(
    lookup='target',
    from_=position_lookup,
    as_=['x2', 'y2']
)

graph_plot = alt.layer(
    edges, nodes, labels,
).configure_view(strokeWidth=0)
graph_plot

alt.LayerChart(...)

In [96]:
def midpoint(edge_id, pos_data):
    '''Given an edge and pos data, returns the midpoint and angle of arrow.'''
    def pos_lookup(node):
        return pos_data[pos_data['node_id'] == edge_datum[node]][['x', 'y']].values[0]
    ((sx, sy), (tx, ty)) = pos_lookup('source'), pos_lookup('target')
    midpoint_x = (sx + tx) / 2 
    midpoint_y = (sy + ty) / 2
    return midpoint_x, midpoint_y



ad = get_arrow_data(edge_data, pos_data)
ad

,midpointx,midpointy,angle
0,-0.299897,-0.680468,64.622508
1,0.423375,0.043162,102.543203
2,0.172184,-0.314673,-171.786328
3,-0.074614,-0.084794,75.054249
4,-0.207980,-0.015959,99.810571
5,0.043210,0.341876,2.368380
6,-0.123026,0.541632,55.778545
7,-0.134268,0.260184,-27.300051
8,0.187818,0.554488,-50.232811
9,0.010340,0.472797,-92.287341


In [97]:
ars = alt.Chart(ad).mark_point(
    shape='triangle',
).encode(
    x=alt.X('midpointx'),
    y=alt.Y('midpointy'),
    angle='angle',
    tooltip=['midpointx', 'midpointy', 'angle']
)

/home/pi/code/envs/graph/lib/python3.8/site-packages/altair/utils/core.py:673: UserWarning: Unrecognized encoding channel 'angle'
  warnings.warn("Unrecognized encoding channel '{}'".format(encoding))


In [247]:
from math import sqrt
idcols = [col for col in edge_data.columns 
 if not any([(col == 'source'), (col == 'target')])]

traildata = edge_data.melt(id_vars=idcols, value_name='node_id', var_name='end')
t = traildata.copy()

def source_target_size(v):
    return 15 if v == 'source' else 1
def get_pos(pos_data, node_id, coord):
    return pos_data[coord][pos_data['node_id']==node_id].values[0]
def get_node(edge_data, edge_id, end):
    return edge_data.query(f'edge_id == {edge_id}')[end].values[0]
def get_arrowhead_base_pos(edge_data, pos_data, edge_id, dt=.1):
    sx = get_pos(pos_data, get_node(edge_data, edge_id, 'source'), 'x')
    sy = get_pos(pos_data, get_node(edge_data, edge_id, 'source'), 'y')
    tx = get_pos(pos_data, get_node(edge_data, edge_id, 'target'), 'x')
    ty = get_pos(pos_data, get_node(edge_data, edge_id, 'target'), 'y')
    d = sqrt((tx - sx)**2 + (ty - sy)**2)
    t = (d-dt)/d
    pt = (1-t)*sx + t*tx, (1-t)*sy + t*ty
    return pt
def transform(row):
    if row.end == 'target':
        row['x'] = get_pos(pos_data, row.node_id, 'x')
        row['y'] = get_pos(pos_data, row.node_id, 'y')
    else: 
        row['x'], row['y'] = get_arrowhead_base_pos(edge_data, pos_data, row.edge_id)
    return row
    

t['s'] = traildata['end'].apply(source_target_size)
m = t.apply(transform, axis=1)
m

,edge_id,words,end,node_id,s,x,y
0,0,"[Vega-Lite, studies, grammar, extents,, which,...",source,A,15,-0.191184,-0.451287
1,1,[],source,B,15,0.375772,0.257119
2,2,"[for, within]",source,B,15,-0.049352,-0.346650
3,3,[a],source,C,15,-0.026693,0.094732
4,4,"[can, synthesizes, low-level]",source,C,15,-0.250595,0.230482
5,5,"[visualization, system, by, range, customized]",source,D,15,0.254140,0.350600
6,6,"[decompose, appearance., reuse, retargeting, e...",source,D,15,-0.034658,0.671557
7,7,"[present, concerned, our, for]",source,D,15,-0.089764,0.237214
8,8,"[first, events]",source,E,15,0.290088,0.431597
9,9,"[novel, an, parameterize, To, us, traditional,...",source,E,15,0.003089,0.291269


In [248]:
trails = alt.Chart(m).mark_trail(
    fillOpacity=0.5,
).encode(
    x="x:Q", 
    y="y:Q", 
    detail='edge_id',    
    size=alt.Size('s', scale=alt.Scale(range=[0,20]), legend=None),
)

In [249]:
graph_plot_trails = alt.layer(
    edges, trails, nodes, labels
).configure_view(strokeWidth=0)
graph_plot_trails

alt.LayerChart(...)

In [234]:
edge_data

,edge_id,source,target,words
0,0,A,C,"[Vega-Lite, studies, grammar, extents,, which,..."
1,1,B,F,[]
2,2,B,C,"[for, within]"
3,3,C,G,[a]
4,4,C,D,"[can, synthesizes, low-level]"
5,5,D,F,"[visualization, system, by, range, customized]"
6,6,D,E,"[decompose, appearance., reuse, retargeting, e..."
7,7,D,G,"[present, concerned, our, for]"
8,8,E,F,"[first, events]"
9,9,E,G,"[novel, an, parameterize, To, us, traditional,..."
